In [1]:
import torch
import torch.nn as nn

**Checking whether mps is available or not**

In [2]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [3]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Create data and send it to the device
mps = torch.rand(size=(3, 4)).to(device)
mps

tensor([[0.1577, 0.3238, 0.7162, 0.5985],
        [0.5257, 0.9904, 0.8671, 0.1598],
        [0.0275, 0.2841, 0.3230, 0.7222]], device='mps:0')

**Constructing Discriminator Network**

In [4]:
class Discriminator(nn.Module):
    
    def __init__(self,channels_img, features_d):
        
        super(Discriminator,self).__init__()
        
        self.disc = nn.Sequential(
            
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1 ),
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d*2, 4, 2, 1),
            self._block(features_d*2, features_d*4, 4, 2, 1),
            self._block(features_d*4, features_d*8, 4, 2, 1),
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            
        )
                
        
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        
        return nn.Sequential(
            
            nn.Conv2d( in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
            
        )
    
    def forward(self, x):
        return self.disc(x)     

**Constructing Generator Network**

In [5]:
class Generator(nn.Module):
    
    def __init__(self, z_dim, channels_img, features_g):
        
        super(Generator, self).__init__()
        
        self.gen = nn.Sequential(
            
            self._block(z_dim, features_g*16, 4, 1, 0), 
            self._block(features_g*16, features_g*8, 4, 2, 1),
            self._block(features_g*8, features_g*4, 4, 2, 1),
            self._block(features_g*4, features_g*2, 4, 2, 1),
            nn.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        
        )
        
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        
        return nn.Sequential(
            
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()            
        
        )   
    
    def forward(self, x):
        return self.gen(x)

In [6]:
def initialize_weights(model):
    
    for m in model.modules():
        if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)
def test():
    N, in_channels, H, W = 8, 3, 64, 64
    z_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)
    
    
    gen = Generator(z_dim, in_channels, 8)
    initialize_weights(gen)
    z = torch.randn((N, z_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W)
    print("Success")

In [7]:
test()

Success


**TRAINING SETUP**

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

Hyperparameters

In [9]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [10]:
LEARNING_RATE = 5e-4
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
CRITIC_ITERATIONS = 5
WEIGHT_CLIP = 0.01

In [11]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

# If you train on MNIST, remember to set channels_img to 1
dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms, download=True
)

 DATA LOADER

In [12]:
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(critic)

OPTIMIZER & CRITERION

In [13]:
opt_gen = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
opt_critic = optim.RMSprop(critic.parameters(), lr=LEARNING_RATE)

In [14]:
fixed_noise = torch.randn(32, Z_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0


In [ ]:
for epoch in range(NUM_EPOCHS):
    
    # Target labels not needed! <3 unsupervised
    
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        
        
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
            fake = gen(noise)
            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake))
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()      
                       
            for p in critic.parameters():
                p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)
                
        output = critic(fake).reshape(-1)
        loss_gen = -torch.mean(output)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        # Print losses occasionally and print to tensorboard
        
        if batch_idx % 200 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1
            gen.train()
            critic.train()


Epoch [0/5] Batch 0/938                   Loss D: -0.6946, loss G: 0.4014


In [ ]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# Assuming your generator model is named `gen` and is already loaded and set to evaluation mode
gen.eval()

# Generate a batch of fake images
fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
with torch.no_grad():
    fake_images = gen(fixed_noise)

# Plotting function
def plot_fake_images(fake_images, num_images=25):
    # Make a grid from the images
    img_grid = vutils.make_grid(fake_images[:num_images], normalize=True, nrow=5)
    img_grid = img_grid.permute(1, 2, 0)  # Change (C, H, W) to (H, W, C) for visualization
    plt.imshow(img_grid.cpu().numpy())
    plt.axis('off')
    plt.title("Generated Images")
    plt.show()

plot_fake_images(fake_images)


In [ ]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# Assuming your generator model is named `gen` and is already loaded and set to evaluation mode
gen.eval()

# Generate a batch of fake images
fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
with torch.no_grad():
    fake_images = gen(fixed_noise)

# Plotting function
def plot_fake_images(fake_images, num_images=32):
    # Make a grid from the images
    img_grid = vutils.make_grid(fake_images[:num_images], normalize=True, nrow=8)  # Adjusted nrow to 4
    img_grid = img_grid.permute(1, 2, 0)  # Change (C, H, W) to (H, W, C) for visualization
    plt.figure(figsize=(10, 10))  # Adjusted figure size for better visualization
    plt.imshow(img_grid.cpu().numpy())
    plt.axis('off')
    plt.title("Generated Images")
    plt.show()

plot_fake_images(fake_images)
